In [11]:
import folium
import folium.plugins
m = folium.Map(arr[0], zoom_start=10, tiles="http://capsule.local:8080/styles/klokantech-basic/{z}/{x}/{y}.png", attr="capsule-maps") 
m

In [21]:
from influxdb import DataFrameClient
import pandas as pd
from datetime import datetime, timedelta

influxdb_client =  DataFrameClient("localhost", "8086", "rudloff", "y4uv3jpc", "telegraf")

#SELECT MEAN("value") FROM "autogen"."mqtt_consumer" 
# WHERE ("topic" = 'gps_measure/latitude') AND time >= '2021-08-05T06:00:00Z'
# AND TIME <= '2021-08-05T06:03:00Z' GROUP BY time(1m) fill(previous)

start = (datetime.strptime("2021-08-05T06:03:00", "%Y-%m-%dT%H:%M:%S") + timedelta(seconds=-5)).isoformat()
end = (datetime.strptime("2021-08-05T06:03:00", "%Y-%m-%dT%H:%M:%S") + timedelta(seconds=5)).isoformat()

# Query the lat and lon values inside of the first and last + margin timestamps
result = influxdb_client.query("SELECT MEAN(\"value\")  FROM \"autogen\".\"mqtt_consumer\" WHERE (\"topic\"\
     = 'gps_measure/latitude') AND time >= '"+start+"Z' AND time <= '"+end+"Z'  GROUP BY time(1m) fill(previous)")
print(result)

ConnectionError: HTTPConnectionPool(host='localhost', port=8086): Max retries exceeded with url: /query?q=SELECT+MEAN%28%22value%22%29++FROM+%22autogen%22.%22mqtt_consumer%22+WHERE+%28%22topic%22+++++%3D+%27gps_measure%2Flatitude%27%29+AND+time+%3E%3D+%272021-08-05T06%3A02%3A55Z%27+AND+time+%3C%3D+%272021-08-05T06%3A03%3A05Z%27++GROUP+BY+time%281m%29+fill%28previous%29&db=telegraf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023A19E0DF08>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

In [ ]:
latitude = pd.DataFrame()
if result:
    latitude = result["mqtt_consumer"]
result = influxdb_client.query("SELECT * FROM \"autogen\".\"mqtt_consumer\" WHERE (\"topic\"\
     = 'gps_measure/longitude') AND time >= '"+start+"Z' AND time <= '"+end+"Z'")
longitude = pd.DataFrame()
if result:
    longitude = result["mqtt_consumer"]
# Check the values
if latitude.empty or longitude.empty:
    return pd.DataFrame()
# Clean, resample 1s just to be sure, add the values to one specific DataFrame
latitude.drop(["host", "topic"], axis=1, inplace=True)
longitude.drop(["host", "topic"], axis=1, inplace=True)
latitude = latitude.resample('1S').first()
longitude = longitude.resample('1S').first()
gps_coords = pd.DataFrame()
gps_coords = latitude.copy()
gps_coords["longitude"] = longitude["value"]
gps_coords.columns = ["latitude", "longitude"]
# Fill NaN cells
gps_coords = gps_coords.fillna(method="ffill")
# Filter out only on timestamps from the images
mask = (gps_coords.index >= timestamps[0]) & (gps_coords.index <= timestamps[-1])
gps_coords = gps_coords.loc[mask]

In [ ]:
path_coords = [[,],[,]]

In [12]:

m2 = folium.Map(arr[1], zoom_start=10)
folium.plugins.AntPath(
    locations=arr, 
    dash_array=[10, 15],
    delay=800,
    weight=6,
    color="#F6FFF3",
    pulse_color="#000000",
    paused=False,
    reverse=False,
).add_to(m2)
m2.fit_bounds(m2.get_bounds())
m2

NameError: name 'arr' is not defined

In [4]:


m3 = folium.Map(arr[1], zoom_start=10)

folium.plugins.AntPath(
    locations=arr, 
    dash_array=[10, 15],
    delay=800,
    weight=6,
    color="#F6FFF3",
    pulse_color="#000000",
    paused=False,
    reverse=False,
    tooltip='<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><img src="road.gif">'
).add_to(m3)
m3.fit_bounds(m3.get_bounds())


marker_cluster = MarkerCluster(name='Photos during the trip').add_to(m3)
marker_cluster_steps = FeatureGroup(name="Trip steps").add_to(m3)
marker_unesco_cluster = MarkerCluster(name='Unesco sites').add_to(m3)

encoded = base64.b64encode(open("moon.jpg", 'rb').read())
html = '<img src="data:image/jpeg;base64,{}">'.format
resolution, width, height = 15, 50, 25
iframe = IFrame(html(encoded.decode('UTF-8')), width=(width*resolution)+20, height=(height*resolution)+20)
popup = folium.Popup(iframe, max_height=1000)

kw = {"prefix": "fa", "color": "green", "icon": "camera"}
# Create photos marker
for i in range(0, len(arr), 200):
    loc = list(arr[i])
    loc[0] += random.uniform(-0.5, 0.5)
    loc[1] += random.uniform(-0.5, 0.5)
    
    encoded = base64.b64encode(open("moon.jpg", 'rb').read())
    html = '<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><p>Coordonnée GPS: {}, {}</p><p><img src="data:image/jpeg;base64,{}"></p>'.format
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, tooltip=html(loc[0], loc[1], encoded.decode('UTF-8'))).add_to(marker_cluster)

kw = {"prefix": "fa", "color": "blue", "icon": "bed"}
# Create sleep steps
for i in range(0, len(arr), 2000):
    loc = list(arr[i])
    html = '<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><p>Coordonnée GPS: {}, {}</p>'.format(loc[0], loc[1])
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, tooltip=html).add_to(marker_cluster_steps)

kw = {"prefix": "fa", "color": "red", "icon": "university"}

for i in range(0, len(unesco_sites)):
    loc = [unesco_sites["latitude"][i], unesco_sites["longitude"][i]]
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, popup=folium.Popup(unesco_sites["short_description_fr"][i], max_width=1000), tooltip=unesco_sites["name_fr"][i]+" \n click for more details").add_to(marker_unesco_cluster)
    
    
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 10px;
    left: 10px;
    width: 250px;
    height: 100px;
    z-index:9999;
    font-size:14px;
    ">
    <p>Temps de voyage: 54 jours</p>
    <p>Kilomètre parcourue: 13141 km</p>
    <p>Pays traversés: 5</p>
    <p>Date de mise à jour: 10 mars 2021</p>
</div>
<div style="
    position: fixed;
    bottom: 0px;
    left: 0px;
    width: 250px;
    height: 120px;
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    opacity: 0.7;
    ">
</div>
{% endmacro %}
"""

legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
m3.get_root().add_child(legend)


folium.LayerControl(collapsed=False).add_to(m3)

plugins.LocateControl().add_to(m3)

plugins.Fullscreen(
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m3)

m3.save("test.html")

m3

NameError: name 'folium' is not defined